# Intake-Postgres Plugin: Basic Usage


### Setup
1. Start a PostgreSQL server. If Docker is installed, an easy way to do this is with the following command:
    ```
    docker run -p 5432:5432 mdillon/postgis:9.6-alpine
    ```
    Wait until the line _"LOG:  database system is ready to accept connections"_ appears.
1. In the same conda environment as this notebook, install `pandas`, `sqlalchemy`, and `psycopg2`. Optionally, `postgresql` can also be installed (this is only the client library, not the database server):
    ```
    conda install pandas sqlalchemy psycopg2 postgres
    ```
1. Install the _intake-postgres_ plugin:
    ```
    conda install -c intake intake-postgres
    ```

### Basic Usage

First, import the necessary modules:

In [1]:
## For inserting test data
import pandas as pd
from sqlalchemy import create_engine

## For using Intake
from intake.catalog import Catalog

Insert some data into database table:

In [2]:
engine = create_engine('postgresql://postgres@localhost:5432/postgres')
df = pd.DataFrame({'first_name': ['Joe', 'Cindy', 'Bob'],
                   'last_name': ['Schmoe', 'Sherman', 'Bullock']},
                 index=[1, 2, 3])
df.to_sql('person', engine)

ValueError: Table 'person' already exists.

Verify the data was written, by connecting to the database directly with the `psql` command-line tool:

In [ ]:
# Verify the data was written
!psql -h localhost -U postgres -c 'select * from person;'

Write out a __basic\_catalog.yml__ file with the appropriate schema:

In [ ]:
%%writefile basic_catalog.yml
plugins:
    source:
        - module: intake_postgres

sources:
    people:
        driver: postgres
        args:
            uri: 'postgresql://postgres@localhost:5432/postgres'
            sql_expr: 'select * from person'
    
    last_names:
        driver: postgres
        args:
            uri: 'postgresql://postgres@localhost:5432/postgres'
            sql_expr: 'select last_name from person'

Access the catalog with Intake:

In [ ]:
catalog = Catalog('basic_catalog.yml')
catalog

Inspect the metadata about the sources (optional):

In [ ]:
catalog['people'].get().discover()

In [ ]:
catalog['last_names'].get().discover()

Read the data from the sources:

In [ ]:
catalog['people'].get().read()

In [ ]:
catalog['last_names'].get().read()